# Main performance experiment for the RandDense models using the new ClimateBench metrics
See Sections 4 and 5 of the manuscript

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/USC Random NN')

Mounted at /content/drive


In [ ]:
!pip install eofs --quiet
import numpy as np
import pandas as pd
from utils import *
results_path = results_path + 'rand_dense/new_metrics_experiment/'

from random_nn import *
import keras
from keras import Model
from keras.layers import *
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import plot_model
tf.get_logger().setLevel('ERROR')
from IPython.display import display

tf.keras.utils.set_random_seed(21)

     |████████████████████████████████| 1.0 MB 15.9 MB/s 


In [ ]:
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']
# Selects first two years of every decade from 1850 onward as validation
val_idx = np.concatenate((np.arange(0,161,10), np.arange(1,162,10),
                          np.arange(165,326,10), np.arange(166,327,10), 
                          np.arange(330,491,10), np.arange(331,492,10),
                          np.arange(500,571,10), np.arange(501,572,10),
                          np.arange(586,657,10), np.arange(587,658,10),
                          np.arange(672,743,10), np.arange(673,744,10)))

# Create training data
train_files = ['historical', 'hist-GHG', 'hist-aer', 'ssp126', 'ssp370', 'ssp585']
X, solvers = create_predictor_data(train_files, sort_by_time=False)
Y = create_predictdand_data(train_files, sort_by_time=False)
original_shape = Y['tas'].shape  # choice of tas here is arbitrary

X_train_dict = {}
Y_train_dict = {}
X_val_dict = {}
Y_val_dict = {}

for var in vars_to_predict:
  Y_var = Y[var].data.reshape((Y[var].shape[0], -1))
  Y_var = pd.DataFrame(Y_var)
    
  X_val = X.iloc[val_idx]
  X_train = X.drop(index=val_idx)
  Y_val = Y_var.iloc[val_idx]
  Y_train = Y_var.drop(index=val_idx)
    
  X_train_dict[var] = X_train
  X_val_dict[var] = X_val
  Y_train_dict[var] = Y_train
  Y_val_dict[var] = Y_val

# Create test data
X_test = get_test_data('ssp245', solvers)
Y_test = create_predictdand_data(['ssp245'])

X_train_dict['tas'].shape, Y_train_dict['tas'].shape, X_val_dict['tas'].shape, Y_val_dict['tas'].shape

((603, 12), (603, 13824), (150, 12), (150, 13824))

In [ ]:
# THESE ARE SOLELY FOR USE IN RESHAPING THE NN OUTPUT LATER
X_test_np = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
X_test_np = X_test_np.sel(time=range(2015,2101))

In [ ]:
param_lims = {1000000: '1M', 10000000: '10M'}
layer_range = (2,11)
num_models = 50
raw_rmse_data_spatial = []
raw_rmse_data_global = []
raw_rmse_data_total = []

In [ ]:
for param_lim in param_lims.keys():
    
  # param_lim +/- 10%
  param_range = (int(param_lim-0.1*param_lim), int(param_lim+0.1*param_lim))
  rmse_data_spatial = []
  rmse_data_global = []
  rmse_data_total = []
  adj_mat_data = []

  for num_layers in range(*layer_range):

    for i in range(num_models):
            
      keras.backend.clear_session()
      untrained_model = None

      model_input = Input(shape=(12,))
      x, adj_mat, param_count = RandDense(prev_layer=model_input,
                                          out_shape=96*144,
                                          layer_count_range=(num_layers,num_layers),
                                          layer_size_range=(model_input.shape[1],96*144),
                                          param_lim=param_range)
      untrained_model = Model(model_input, x)

      #untrained_model.summary()
      #display(plot_model(untrained_model, show_shapes=True))

      np.save(results_path+f'{param_lims[param_lim]}/models/{num_layers}_layer_model_{i}', untrained_model.get_config())
      plot_model(untrained_model, results_path+f'{param_lims[param_lim]}/images/{num_layers}_layer_model_{i}.png', show_shapes=True)
      adj_mat_data.append(adj_mat)
            
      for var in vars_to_predict:
                
        # Get train/val data
        X_train = X_train_dict[var]
        Y_train = Y_train_dict[var]
        X_val = X_val_dict[var]
        Y_val = Y_val_dict[var]

        model = None
        model = untrained_model
        model.compile(optimizer="adam", loss="mse", metrics=["mse"])
        hist = model.fit(X_train,
                         Y_train,
                         batch_size=16,
                         epochs=100,
                         verbose=0,
                         validation_data=(X_val,Y_val),
                         callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
                        )
        #plot_loss(hist)
        
        # Make predictions using trained model
        m_pred = model.predict(X_test, verbose=0)
        m_pred = m_pred.reshape((-1,) + original_shape[1:])
        m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords= [X_test_np.time.data, X_test_np.latitude.data, X_test_np.longitude.data ])
        m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)

        # Save prediction as .nc
        if var == 'diurnal_temperature_range':
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/dtr/{num_layers}_layer_model_{i}.nc', 'w')
        else:
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/{var}/{num_layers}_layer_model_{i}.nc', 'w')

        # Calculate RMSE
        var_truth = Y_test[var]
        m_var_pred = m_pred.transpose('time','lat','lon')[var]
        rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_spatial.append(rmse_spatial)
        rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_global.append(rmse_global)
        rmse_total = rmse_spatial + 5*rmse_global
        raw_rmse_data_total.append(rmse_total)
                
    n_layer_rmse_data_spatial = raw_rmse_data_spatial[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_spatial = np.reshape(n_layer_rmse_data_spatial, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_global = raw_rmse_data_global[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_global = np.reshape(n_layer_rmse_data_global, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_total = raw_rmse_data_total[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_total = np.reshape(n_layer_rmse_data_total, (len(vars_to_predict), num_models), order='F')

    print(f'{num_layers} Layer {param_lims[param_lim]} Mean RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')

    print(f'{num_layers} Layer {param_lims[param_lim]} Min RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
            
    rmse_data_spatial.append(n_layer_rmse_data_spatial)
    rmse_data_global.append(n_layer_rmse_data_global)
    rmse_data_total.append(n_layer_rmse_data_total)
        
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_spatial', rmse_data_spatial)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_global', rmse_data_global)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_total', rmse_data_total)
  adj_mat_data = np.array(adj_mat_data, dtype='object')
  adj_mat_data = np.reshape(adj_mat_data, (len(range(*layer_range)), num_models))
  np.save(results_path+f'{param_lims[param_lim]}/adj_mat_data', adj_mat_data)

2 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.4321
		Global: 0.3764
		Total: 2.3142
	dtr:
		Spatial: 12.0842
		Global: 1.7575
		Total: 20.872
	pr:
		Spatial: 3.718
		Global: 0.4945
		Total: 6.1904
	pr90:
		Spatial: 4.3762
		Global: 0.6095
		Total: 7.4236
2 Layer 1M Min RMSE:
	tas:
		Spatial: 0.3012
		Global: 0.2622
		Total: 1.6122
	dtr:
		Spatial: 9.9182
		Global: 0.8831
		Total: 14.6711
	pr:
		Spatial: 2.9747
		Global: 0.2657
		Total: 4.4724
	pr90:
		Spatial: 3.6362
		Global: 0.4785
		Total: 6.176
3 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.4568
		Global: 0.401
		Total: 2.4616
	dtr:
		Spatial: 12.6596
		Global: 2.1536
		Total: 23.4277
	pr:
		Spatial: 3.6854
		Global: 0.5811
		Total: 6.5908
	pr90:
		Spatial: 4.3316
		Global: 0.6568
		Total: 7.6154
3 Layer 1M Min RMSE:
	tas:
		Spatial: 0.3445
		Global: 0.302
		Total: 1.8547
	dtr:
		Spatial: 10.6897
		Global: 0.8153
		Total: 15.5233
	pr:
		Spatial: 2.7295
		Global: 0.3523
		Total: 4.9406
	pr90:
		Spatial: 3.4308
		Global: 0.5099
		Total: 5.9804
